In [585]:

import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import panel as pn
from panel import widgets
from panel import interact
import plotly.express as px
from pathlib import Path
import numpy as np
from datetime import datetime, timedelta
import alpaca_trade_api as tradeapi
import os
import seaborn as sns
from dotenv import load_dotenv
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA

%matplotlib inline

# Initialize the Panel Extensions (for Plotly)
pn.extension('plotly')


In [586]:
# Read the API key

load_dotenv()

APCA_API_KEY_ID = os.getenv("Api_key")
APCA_SECRET_KEY = os.getenv("Secret_key")

api = tradeapi.REST(APCA_API_KEY_ID, APCA_SECRET_KEY, api_version='v2')

type(APCA_API_KEY_ID)



str

In [589]:
# Import Data from alpha vantage, may have to wait a minute between cells.

materials = api.alpha_vantage.historic_quotes('XLB', adjusted=True, output_format='pandas')

industrials = api.alpha_vantage.historic_quotes('XLI', adjusted=True, output_format='pandas')

technology = api.alpha_vantage.historic_quotes('XLK', adjusted=True, output_format='pandas')

real_estate = api.alpha_vantage.historic_quotes('RWR', adjusted=True, output_format='pandas')

staples = api.alpha_vantage.historic_quotes('XLP', adjusted=True, output_format='pandas')
staples.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient
date,,,,,,,,
2020-05-01,57.94,58.43,57.38,57.62,57.62,13023630.0,0.0,1.0
2020-04-30,58.79,58.82,57.92,58.26,58.26,16396307.0,0.0,1.0
2020-04-29,59.51,59.51,58.40,58.96,58.96,16038513.0,0.0,1.0
2020-04-28,59.56,59.64,58.96,59.21,59.21,13608799.0,0.0,1.0
2020-04-27,59.07,59.15,58.65,58.88,58.88,11024009.0,0.0,1.0


In [590]:
# Import Data from alpha vantage, may have to wait a minute between cells.

cons_disc = api.alpha_vantage.historic_quotes('XLY', adjusted=True, output_format='pandas')

utilities = api.alpha_vantage.historic_quotes('XLU', adjusted=True, output_format='pandas')

financials = api.alpha_vantage.historic_quotes('XLF', adjusted=True, output_format='pandas')

transports = api.alpha_vantage.historic_quotes('IYT', adjusted=True, output_format='pandas')

health = api.alpha_vantage.historic_quotes('XLV', adjusted=True, output_format='pandas')


In [591]:
# Import Data from alpha vantage, may have to wait a minute between cells.

energy = api.alpha_vantage.historic_quotes('XLE', adjusted=True, output_format='pandas')

spy = api.alpha_vantage.historic_quotes('SPY', adjusted=True, output_format='pandas')


In [592]:
#Set up data for backtesting

materials.rename(columns={'1. open':'Open','2. high':'High','3. low':'Low', '5. adjusted close':'Close', '6. volume':'Volume'}, inplace=True)
materials.drop(columns=['4. close','7. dividend amount', '8. split coefficient'], inplace=True)
materials.sort_index(ascending=True, inplace=True)

industrials.rename(columns={'1. open':'Open','2. high':'High','3. low':'Low', '5. adjusted close':'Close', '6. volume':'Volume'}, inplace=True)
industrials.drop(columns=['4. close','7. dividend amount', '8. split coefficient'], inplace=True)
industrials.sort_index(ascending=True, inplace=True)

technology.rename(columns={'1. open':'Open','2. high':'High','3. low':'Low', '5. adjusted close':'Close', '6. volume':'Volume'}, inplace=True)
technology.drop(columns=['4. close','7. dividend amount', '8. split coefficient'], inplace=True)
technology.sort_index(ascending=True, inplace=True)

real_estate.rename(columns={'1. open':'Open','2. high':'High','3. low':'Low', '5. adjusted close':'Close', '6. volume':'Volume'}, inplace=True)
real_estate.drop(columns=['4. close','7. dividend amount', '8. split coefficient'], inplace=True)
real_estate.sort_index(ascending=True, inplace=True)

staples.rename(columns={'1. open':'Open','2. high':'High','3. low':'Low', '5. adjusted close':'Close', '6. volume':'Volume'}, inplace=True)
staples.drop(columns=['4. close','7. dividend amount', '8. split coefficient'], inplace=True)
staples.sort_index(ascending=True, inplace=True)

cons_disc.rename(columns={'1. open':'Open','2. high':'High','3. low':'Low', '5. adjusted close':'Close', '6. volume':'Volume'}, inplace=True)
cons_disc.drop(columns=['4. close','7. dividend amount', '8. split coefficient'], inplace=True)
cons_disc.sort_index(ascending=True, inplace=True)

utilities.rename(columns={'1. open':'Open','2. high':'High','3. low':'Low', '5. adjusted close':'Close', '6. volume':'Volume'}, inplace=True)
utilities.drop(columns=['4. close','7. dividend amount', '8. split coefficient'], inplace=True)
utilities.sort_index(ascending=True, inplace=True)

financials.rename(columns={'1. open':'Open','2. high':'High','3. low':'Low', '5. adjusted close':'Close', '6. volume':'Volume'}, inplace=True)
financials.drop(columns=['4. close','7. dividend amount', '8. split coefficient'], inplace=True)
financials.sort_index(ascending=True, inplace=True)

transports.rename(columns={'1. open':'Open','2. high':'High','3. low':'Low', '5. adjusted close':'Close', '6. volume':'Volume'}, inplace=True)
transports.drop(columns=['4. close','7. dividend amount', '8. split coefficient'], inplace=True)
transports.sort_index(ascending=True, inplace=True)

health.rename(columns={'1. open':'Open','2. high':'High','3. low':'Low', '5. adjusted close':'Close', '6. volume':'Volume'}, inplace=True)
health.drop(columns=['4. close','7. dividend amount', '8. split coefficient'], inplace=True)
health.sort_index(ascending=True, inplace=True)

energy.rename(columns={'1. open':'Open','2. high':'High','3. low':'Low', '5. adjusted close':'Close', '6. volume':'Volume'}, inplace=True)
energy.drop(columns=['4. close','7. dividend amount', '8. split coefficient'], inplace=True)
energy.sort_index(ascending=True, inplace=True)

spy.rename(columns={'1. open':'Open','2. high':'High','3. low':'Low', '5. adjusted close':'Close', '6. volume':'Volume'}, inplace=True)
spy.drop(columns=['4. close','7. dividend amount', '8. split coefficient'], inplace=True)
spy.sort_index(ascending=True, inplace=True)
staples.head()


,Open,High,Low,Close,Volume
date,,,,,
2000-05-01,22.50,22.81,22.16,14.2400,21200.0
2000-05-02,22.59,22.78,22.39,14.2778,19000.0
2000-05-03,22.94,22.97,22.50,14.2778,309800.0
2000-05-04,22.75,22.94,22.52,14.2778,30600.0
2000-05-05,22.63,23.22,22.63,14.6113,11600.0


In [675]:
# Define Panel Visualization Functions
#Function to create backtesting plot
def backtest_sp500():
    class SmaCross(Strategy):
        n1 =10
        n2 = 75
        def init(self):
            Close = self.data.Close
            self.ma1 = self.I(SMA, Close,self.n1)
            self.ma2 = self.I(SMA, Close, self.n2)

        def next(self):
            if crossover(self.ma1, self.ma2):
                self.buy()
            elif crossover(self.ma2, self.ma1):
                self.sell()

        
                
    bt_spy = Backtest(spy, SmaCross,
                              cash=10000, commission=.002)
    
#     stats = bt_spy.optimize(n1=range(5, 30, 5),
#                     n2=range(10, 365, 5),
#                     maximize='Equity Final [$]',
#                     constraint=lambda p: p.n1 < p.n2)
   

    bt_spy.run()
    bt_spy_plot = bt_spy.plot()
    #return bt_materials_plot
    return bt_spy.run()


In [676]:
# Define Panel Visualization Functions
#Function to create backtesting plot
def backtest_materials():
    class SmaCross(Strategy):
        
        n1 =10
        n2 =360
        
        def init(self):
            Close = self.data.Close
            self.ma1 = self.I(SMA, Close, self.n1)
            self.ma2 = self.I(SMA, Close, self.n2)

        def next(self):
            if crossover(self.ma1, self.ma2):
                self.buy()
            elif crossover(self.ma2, self.ma1):
                self.sell()


    bt_materials = Backtest(materials_sector, SmaCross,
                              cash=10000, commission=.002)
#     stats = bt_materials.optimize(n1=range(5, 30, 5),
#                     n2=range(10, 365, 5),
#                     maximize='Equity Final [$]',
#                     constraint=lambda p: p.n1 < p.n2)   
    
    bt_materials.run()
    #bt_materials_plot = bt_materials.plot()
 
    return bt_materials.run()

In [677]:
def backtest_industrials():
    class SmaCross(Strategy):
        
        n1=5
        n2=250
        def init(self):
            Close = self.data.Close
            self.ma1 = self.I(SMA, Close, self.n1)
            self.ma2 = self.I(SMA, Close, self.n2)

        def next(self):
            if crossover(self.ma1, self.ma2):
                self.buy()
            elif crossover(self.ma2, self.ma1):
                self.sell()


    bt_industrials = Backtest(industrials_sector, SmaCross,
                              cash=10000, commission=.002)
    
#     stats = bt_industrials.optimize(n1=range(5, 30, 5),
#                     n2=range(10, 365, 5),
#                     maximize='Equity Final [$]',
#                     constraint=lambda p: p.n1 < p.n2)   
    bt_industrials.run()
    #bt_industrials_plot = bt_industrials.plot()
            
    return bt_industrials.run()


In [678]:
def backtest_technology():
    class SmaCross(Strategy):
        
        n1=10
        n2=95
        def init(self):
            Close = self.data.Close
            self.ma1 = self.I(SMA, Close, self.n1)
            self.ma2 = self.I(SMA, Close, self.n2)

        def next(self):
            if crossover(self.ma1, self.ma2):
                self.buy()
            elif crossover(self.ma2, self.ma1):
                self.sell()


    bt_technology = Backtest(technology, SmaCross,
                              cash=10000, commission=.002)
    
#     stats = bt_technology.optimize(n1=range(5, 30, 5),
#                     n2=range(10, 365, 5),
#                     maximize='Equity Final [$]',
#                     constraint=lambda p: p.n1 < p.n2)   
    bt_technology.run()
    #bt_technology_plot = bt_technology.plot()
            
    return bt_technology.run()

In [679]:
def backtest_real_estate():
    class SmaCross(Strategy):
        n1=20
        n2=50
        def init(self):
            Close = self.data.Close
            self.ma1 = self.I(SMA, Close, self.n1)
            self.ma2 = self.I(SMA, Close, self.n2)

        def next(self):
            if crossover(self.ma1, self.ma2):
                self.buy()
            elif crossover(self.ma2, self.ma1):
                self.sell()


    bt_real_estate = Backtest(real_estate, SmaCross,
                              cash=10000, commission=.002)
    
    
#     stats = bt_real_estate.optimize(n1=range(5, 30, 5),
#                     n2=range(10, 365, 5),
#                     maximize='Equity Final [$]',
#                     constraint=lambda p: p.n1 < p.n2)     
    
    bt_real_estate.run()
    #bt_real_estate_plot = bt_real_estate.plot()
            
    return bt_real_estate.run()

In [680]:
def backtest_staples():
    class SmaCross(Strategy):
        n1=25
        n2=355
        def init(self):
            Close = self.data.Close
            self.ma1 = self.I(SMA, Close, self.n1)
            self.ma2 = self.I(SMA, Close, self.n2)

        def next(self):
            if crossover(self.ma1, self.ma2):
                self.buy()
            elif crossover(self.ma2, self.ma1):
                self.sell()


    bt_staples = Backtest(staples, SmaCross,
                          cash=10000, commission=.002)
    
    
#     stats = bt_staples.optimize(n1=range(5, 30, 5),
#                     n2=range(10, 365, 5),
#                     maximize='Equity Final [$]',
#                     constraint=lambda p: p.n1 < p.n2)     
    
    bt_staples.run()
    #bt_real_estate_plot = bt_real_estate.plot()
            
    return bt_staples.run()

In [681]:
def backtest_cons_disc():
    class SmaCross(Strategy):
        n1=5
        n2=220
        def init(self):
            Close = self.data.Close
            self.ma1 = self.I(SMA, Close, self.n1)
            self.ma2 = self.I(SMA, Close, self.n2)

        def next(self):
            if crossover(self.ma1, self.ma2):
                self.buy()
            elif crossover(self.ma2, self.ma1):
                self.sell()


    bt_cons_disc = Backtest(cons_disc, SmaCross,
                          cash=10000, commission=.002)
    
    
#     stats = bt_cons_disc.optimize(n1=range(5, 30, 5),
#                     n2=range(10, 365, 5),
#                     maximize='Equity Final [$]',
#                     constraint=lambda p: p.n1 < p.n2)     
    
    bt_cons_disc.run()
    #bt_real_estate_plot = bt_real_estate.plot()
            
    return bt_cons_disc.run()

In [682]:
def backtest_utilities():
    class SmaCross(Strategy):
        n1=25
        n2=355
        def init(self):
            Close = self.data.Close
            self.ma1 = self.I(SMA, Close, self.n1)
            self.ma2 = self.I(SMA, Close, self.n2)

        def next(self):
            if crossover(self.ma1, self.ma2):
                self.buy()
            elif crossover(self.ma2, self.ma1):
                self.sell()


    bt_utilities = Backtest(utilities, SmaCross,
                          cash=10000, commission=.002)
    
    
    stats = bt_utilities.optimize(n1=range(5, 30, 5),
                    n2=range(10, 365, 5),
                    maximize='Equity Final [$]',
                    constraint=lambda p: p.n1 < p.n2)     
    
    bt_utilities.run()
    #bt_real_estate_plot = bt_real_estate.plot()
            
    return bt_utilities.run()

In [683]:
def backtest_financials():
    class SmaCross(Strategy):
        n1=15
        n2=125
        def init(self):
            Close = self.data.Close
            self.ma1 = self.I(SMA, Close, self.n1)
            self.ma2 = self.I(SMA, Close, self.n2)

        def next(self):
            if crossover(self.ma1, self.ma2):
                self.buy()
            elif crossover(self.ma2, self.ma1):
                self.sell()


    bt_financials = Backtest(financials, SmaCross,
                          cash=10000, commission=.002)
    
    
#     stats = bt_financials.optimize(n1=range(5, 30, 5),
#                     n2=range(10, 365, 5),
#                     maximize='Equity Final [$]',
#                     constraint=lambda p: p.n1 < p.n2)     
    
    bt_financials.run()
    #bt_real_estate_plot = bt_real_estate.plot()
            
    return bt_financials.run()

In [688]:
def backtest_transports():
    class SmaCross(Strategy):
        n1=25
        n2=155
        def init(self):
            Close = self.data.Close
            self.ma1 = self.I(SMA, Close, self.n1)
            self.ma2 = self.I(SMA, Close, self.n2)

        def next(self):
            if crossover(self.ma1, self.ma2):
                self.buy()
            elif crossover(self.ma2, self.ma1):
                self.sell()


    bt_transports = Backtest(transports, SmaCross,
                          cash=10000, commission=.002)
    
    
#     stats = bt_transports.optimize(n1=range(5, 30, 5),
#                     n2=range(10, 365, 5),
#                     maximize='Equity Final [$]',
#                     constraint=lambda p: p.n1 < p.n2)     
    
    bt_transports.run()
    #bt_real_estate_plot = bt_real_estate.plot()
            
    return bt_transports.run()

In [689]:
# Create panels to structure the layout of the dashboard


column1 = pn.Column(
    "## S&P 500 backtest", backtest_sp500(), 
   
)

column2 = pn.Column(
    "## Materials backtest", backtest_materials(), 
   
)

column3 = pn.Column(
      "## Industrials backtest", backtest_industrials(), 
   
 )

column4 = pn.Column(
        "## Technology backtest", backtest_technology(), 
)
 
column5 = pn.Column(
        "## Real Estate backtest", backtest_real_estate(), 
)

column6 = pn.Column(
        "## Staples backtest", backtest_staples(), 
)

column7 = pn.Column(
        "## Consumer Discretionary backtest", backtest_cons_disc(), 
)

column8 = pn.Column(
        "## Utilities backtest", backtest_utilities(), 
)

column9 = pn.Column(
        "## Financials backtest", backtest_financials(),     
)

column10 = pn.Column(
        "## Transports backtest", backtest_transports(),     
)

# column11 = pn.Column(
#         "## Health backtest", backtest_health(),     
# )

# column12 = pn.Column(
#         "## Energy backtest", backtest_energy(),     
# )


# column4 = pn.Column(
#     "## Parallel Coordinates and Parallel Categories",
#     create_parallel_coordinates(),
#     create_parallel_categories(),
# )

# Create tabs


sector_dashboard = pn.Tabs(
    ("SP 500", column1), ("Materials", column2), ("Industrials", column3), ("Technology", column4),
    ("Real Estate", column5),("Staples",column6), ("Consumer Discretionary", column7),
    ("Utilities", column8),("Financials", column9),("Transports", column10),("Health", column10),
    ("Energy", column10)
)  
    
    #("Avg Sales Prices", column2), ("Top 10 Neighborhoods and Map", column3),
    #("Parallels", column4)
    


C:\Users\costa\Anaconda3\envs\vizenv\lib\site-packages\ipykernel_launcher.py:24: UserWarning:

Searching best of 345 configurations.



In [690]:
#Serve the dashboard

sector_dashboard.servable()


Tabs
    [0] Column
        [0] Markdown(str)
        [1] DataFrame(Series)
    [1] Column
        [0] Markdown(str)
        [1] DataFrame(Series)
    [2] Column
        [0] Markdown(str)
        [1] DataFrame(Series)
    [3] Column
        [0] Markdown(str)
        [1] DataFrame(Series)
    [4] Column
        [0] Markdown(str)
        [1] DataFrame(Series)
    [5] Column
        [0] Markdown(str)
        [1] DataFrame(Series)
    [6] Column
        [0] Markdown(str)
        [1] DataFrame(Series)
    [7] Column
        [0] Markdown(str)
        [1] DataFrame(Series)
    [8] Column
        [0] Markdown(str)
        [1] DataFrame(Series)
    [9] Column
        [0] Markdown(str)
        [1] DataFrame(Series)

In [ ]:
# Wrap Plotly object by explicitly declaring Panel pane
# pane = pn.pane.Plotly(plot)
# pane

In [ ]:
# Wrap Plotly object by using panel.panel helper function
#pn.panel(plot)

In [ ]:
# Print the type of object
#pane.pprint()

In [ ]:
# Create row
# row = pn.Row(scatter_plot, bar_plot)
# row

In [ ]:
# Create column using Markdown and row object
# column = pn.Column(
#     '# Allegheny, PA Real Estate Visualizations',
#     '## *Sales and Foreclosures*',
#     row)
# column

In [ ]:
# Put bar plots in row
# row_of_bar = pn.Row(num_foreclosures_plot)
# row_of_bar.append(num_sales_plot)

In [ ]:
# Put parallel plots in a single row
#row_of_parallel = pn.Row(parallel_categories, parallel_coordinates)

In [ ]:
# Insert row_of_parallel and row_of_Bar into a column object with Markdown text
# plots_as_column = pn.Column(
#     "# Allegheny Real Estate Dashboard", row_of_parallel, row_of_bar
# )

In [ ]:
# Create tabs
## Create tabs
# tabs = pn.Tabs(
#     ("All Plots", plots_as_column),
#     ("General Plots", row_of_bar),
#     ("Statistical Plots", row_of_parallel)
# )
# tabs